In [1]:
from collections import defaultdict
import networkx as nx
#from Exceptions import ValueError
from random import sample
from itertools import combinations
from math import log

from scipy.stats import entropy 
from phoenix.helpers import *
from phoenix.SortedKeyCollections import *

import numpy as np


def jsd2(dist1, dist2, i=None, debug=False): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)

    x = []
    y = []
    if len(dist1) < len(dist2):
        #x = np.append(x, [0 for r in range(dif)])
        for key, val in dist2.items():
            x.append(dist1.get(key, 0))
            y.append(val)
            
    elif len(dist1) >= len(dist2): 
        #y = np.append(y, [0 for r in range(dif)])
        #x = np.array(x)
        for key, val in dist1.items():
            x.append(val)
            y.append(dist2.get(key, 0))
            

    x = np.array(x)
    y = np.array(y)
    #print(x,y)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    #print(x, y)
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*np.sum(d1+d2)    
    return d

def avg_jsd2(dists1, dists2, i=None):
    ttl_js_div = sum([jsd2(dist1, dists2.get(key, {}), i) 
                      for key, dist1 in dists1.items()])
    
    if i: print(i)
    
    return ttl_js_div/len(dists1)

def decompress_generate(ref_clq_num_dist, ref_clq2clq_dist, node_count, kl_threshold, iteration_cap=100, debug=False):
    """In the greedy step (decision process), we choose the operation
    that most minimizes the kl divergence."""
    nodes = list(range(node_count))
    ### Initialize clqs with a clique of random selected size 
    clqs = []
    clq_size = get_weighted_random_value(ref_clq_num_dist)
    clq_nodes = sample(nodes, clq_size)
    clqs.append(clq_nodes)
    ### Initialize hypergraph's distributions
    clq_num_dist = clique_number_distribution(clqs)
    clq2clq_dist = cliques_x_cliques_distribution(clqs)
    ### Initialize kl divergence scores
    avg_js = 100
    ###
    i = 0
    
    if debug: print("i | clique-number kl | clqXclq intxn size kl")
    while avg_js > kl_threshold and i < iteration_cap:
        if debug:
            if i%10==0: 
                print(i)
        # climb down from arbitrarily high KL value
        min_clq_num_js = 100
        min_clq2clq_js = 100
        min_clq = []
        
        ### Fully Exhaustive Greedy Decision Process (GDP) ###
        for r in range(1, max(ref_clq_num_dist)+1):
            #random_clq_of_size_r = sample(list(combinations(nodes, r)), 1)
            #assert len(random_clq_of_size_r[0]) > 0
            if debug: print(i, r)
            for potential_next_clq in combinations(nodes, r):
                if debug: print(potential_next_clq)
                # update dists to represent one step forward
                new_clq_num_dist = normalize_distribution(clique_number_distribution(clqs+[potential_next_clq]))
                new_clq2clq_dist = normalize_distributions(cliques_x_cliques_distribution(clqs+[potential_next_clq]))
            
                # calculate the kl divergences of next step
                js_1 = jsd2(ref_clq_num_dist, new_clq_num_dist)
                js_2 = avg_jsd2(ref_clq2clq_dist, new_clq2clq_dist)
                
                # if debug: print("jsd:", js_1, "avg_jsd:", js_2)
                    
                ## king of the valley(?)-type of minimization
                #if (kl_1 < min_clq_num_js and
                #    kl_2 < min_clq_x_clq_js):
                    
                if (js_1+js_2 <= min_clq_num_js + min_clq2clq_js):
                    min_clq_num_js = js_1
                    min_clq2clq_js = js_2
                    min_clq = potential_next_clq
                    
                    #if debug: print("jsd:", js_1, "avg_jsd:", js_2)
        ######### End of GDP ########
        
        ### Update list of cliques, reference models, and average KL score
        if min_clq not in clqs:
            clqs.append(list(min_clq))
            yield min_clq
            
            clq_num_dist = normalize_distribution(clique_number_distribution(clqs))
            clq2clq_dist = normalize_distributions(cliques_x_cliques_distribution(clqs))
            
            assert min_clq_num_js == jsd2(ref_clq_num_dist, clq_num_dist)
            assert min_clq2clq_js == avg_jsd2(ref_clq2clq_dist, clq2clq_dist)
            
            avg_js = (min_clq_num_js+min_clq2clq_js)/2

            if debug:
                print(i, "|", min_clq_num_js, "|", min_clq2clq_js)                    
                print()
        i += 1
        
    yield clqs
    

def generate_edges_from_clique(nodes):
    from itertools import combinations
    for c in combinations(nodes, 2):
        yield c

In [2]:
%matplotlib inline

from networkx.generators import watts_strogatz_graph, barabasi_albert_graph

#graph = nx.Graph([[0,1], [1,2], [2,3], [1,3], [1,4], [2,6], [0,2]]); graph.add_node(5)
graph = nx.read_gml('../demo_graphs/karate_Zachary.gml')
#graph = barabasi_albert_graph(15, 2)

node_count = graph.number_of_nodes() 
clqs = list(nx.find_cliques(graph))
nx.draw_networkx(graph)

NetworkXError: cannot tokenize u'graph' at (3, 1)

In [ ]:
clq_num_dist = normalize_distribution(clique_number_distribution(clqs))

clq2clq_dist = normalize_distributions(cliques_x_cliques_distribution(clqs))

d = list(decompress_generate(clq_num_dist, clq2clq_dist, node_count, 0.5, iteration_cap=36, debug=True))

gen_clqs = d[-1]
model1 = normalize_distribution(clique_number_distribution(gen_clqs))
model2 = normalize_distributions(cliques_x_cliques_distribution(gen_clqs))

i | clique-number kl | clqXclq intxn size kl
0
(0, 1)
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)
(24,)
(25,)
(26,)
(27,)
(28,)
(29,)
(30,)
(31,)
(32,)
(33,)
(0, 2)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(2, 16)
(2, 17)
(2, 18)
(2, 19)
(2, 20)
(2, 21)
(2, 22)
(2, 23)
(2, 24)
(2, 25)
(2, 26)
(2, 27)
(2, 28)
(2, 29)
(2, 30)
(2, 31)
(2, 32)
(2, 33)
(

In [ ]:
gen_edges = [e for c in gen_clqs for e in generate_edges_from_clique(c)]

gen_graph = nx.Graph(gen_edges)

for r in range(node_count):
    gen_graph.add_node(r)

nx.draw_networkx(gen_graph)

In [ ]:
from scipy.stats import entropy 

ref_dist = {(2, 2): Counter({0: 0.9, 1: 0.1}),
 (2, 3): Counter({0: 0.8, 1: 0.2}),
 (2, 4): Counter({0: 0.7, 1: 0.3})}

# this dist is perfect as each internal distribution 
# is of the same key size (feature(?), length)
good_dist = {(2, 2): Counter({0: 0.5, 1: 0.5}),
 (2, 3): Counter({0: 0.5, 1: 0.5}),
 (2, 4): Counter({0: 0.9, 1: 0.1})}

# this dist breaks bc one internal dist is of diff. size
# to its respective internal reference dist.
bad_dist = {(2, 2): Counter({0: 0.7, 1: 0.3}),
            (2, 3): Counter({0: 0.1, 1: 0.5, 2:0.4}), 
            (2, 4): Counter({0:1.0})
           } # breaks here

#print(avg_kl_div(ref_dist, good_dist))

print(avg_jsd(bad_dist, ref_dist))